Model:

$\mu = m_B^{*}-\left(M_B-\alpha\times X_1 +\beta\times C\right)$

$\mu$: distance modulus

$m_B^{*}$: peak magnitude in rest frame

$M_B$: absolute magnitude

$\alpha, \beta$: bias parameters

$C$: color

In [19]:
import glob
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from astropy.cosmology import FlatLambdaCDM
import emcee
from astropy import units as u
import seaborn as sns

In [26]:
def c_stat(n): #Número de SN
    c00 = loadsqmat('jla_v0_covmatrix.dat')
    c11 = loadsqmat('jla_va_covmatrix.dat')
    c22 = loadsqmat('jla_vb_covmatrix.dat')
    c01 = loadsqmat('jla_v0a_covmatrix.dat')
    c02 = loadsqmat('jla_v0b_covmatrix.dat')
    c12 = loadsqmat('jla_vab_covmatrix.dat')
    c = np.zeros((3 * n, 3 * n))
    
    for i in range(n):
        for j in range(n):
            c[3 * i + 2, 3 * j + 2] = c00[i, j]
            c[3 * i + 1, 3 * j + 1] = c11[i, j]
            c[3 * i, 3 * j] = c22[i, j]
    
            c[3 * i + 2, 3 * j + 1] = c01[i, j]
            c[3 * i + 2, 3 * j] = c02[i, j]
            c[3 * i, 3 * j + 1] = c12[i, j]
    
            c[3 * j + 1, 3 * i + 2] = c01[i, j]
            c[3 * j, 3 * i + 2] = c02[i, j]
            c[3 * j + 1, 3 * i] = c12[i, j]

    return c

def mu_cov(alpha, beta,Ceta):
    Cmu = np.zeros_like(Ceta[::3, ::3])
    
    coefficients = [1., alpha, -beta]
    for i, coef1 in enumerate(coefficients):
        for j, coef2 in enumerate(coefficients):
            Cmu += (coef1 * coef2) * Ceta[i::3, j::3]
    
    sigma = np.loadtxt('./sigma_mu.txt')
    sigma_pecvel = (5 * 150 / 3e5) / (np.log(10.) * sigma[:, 2])
    Cmu[np.diag_indices_from(Cmu)] += sigma[:, 0] ** 2 + sigma[:, 1] ** 2 + sigma_pecvel ** 2
    
    return Cmu



def A_matrix(alpha,beta):
    I = np.identity(740)
    a_vec = (1,alpha,-beta)
    A = np.tensordot(I,a_vec,axes=0).reshape((740,2220))

    return A

def eta_matrix(X_1,C,m_b):
    eta = np.zeros((3*740))
    for i in range(740):
        eta[3*i] = X_1[i]
        eta[3*i +1] = C[i]
        eta[3*i +2] = m_b[i]
    return eta


def lumdist(z, Om):
    cosmo = FlatLambdaCDM(H0=70, Om0=Om, Tcmb0=2.725)

    return  cosmo.luminosity_distance(z)

def lumvec(stellar_mass):
    dM_B = np.ones_like(stellar_mass)
    for i in range(len(stellar_mass)):
        if stellar_mass[i]<10:
            dM_B[i] = 0
        
    return dM_B     

def log_likelihood(theta, lumvec, eta, z,C_eta):
    M_B, dM_B, alpha, beta, Om = theta #Varied parameters
    A = A_matrix(alpha,beta)
    cov = mu_cov(alpha,beta,C_eta)
    covariance = cov
    model =np.matmul(A,eta)-(M_B * np.ones(740)+ dM_B * lumvec) #Model
    mu = 5 * np.log10(lumdist(z, Om).to_value(u.Mpc)) + 25
    vec = (model - mu) #Data vector
    Xi = vec.T @ np.linalg.solve(np.linalg.inv(cov), vec)  
    
    return -0.5*Xi

def log_prior(theta):
    M_B, dM_B, alpha, beta, Om = theta
    if -20 < M_B < -18 and -1.0 < dM_B < 0 and 0 < alpha < 0.5 and 2.0 < beta < 4.0 and 0.25 < Om < 0.35: #priors based on https://arxiv.org/pdf/1401.4064 best fit values
        return 0.0
    return -np.inf

def log_probability(theta,lumvec, eta, z,C_eta):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, lumvec, eta, z,C_eta)


In [21]:
#Data
import pandas as pd
data = pd.read_csv('jla_likelihood_v6/jla_likelihood_v6/data/jla_lcparams.txt', sep='\s+')
C_eta = sum([fits.getdata(mat) for mat in glob.glob('C*.fits')])
luvec = lumvec(data['3rdvar']) #Vector de \Delta{M_B}
eta = eta_matrix(data['x1'],data['color'],data['mb'])
z = data['zcmb']

In [43]:
ndim = 5
nwalkers = 50
p0 = np.zeros((20,5))

mean =[-19,-0.05,0.1,3,0.3]
std = [0.01,0.01, 0.01, 0.01, 0.01]  
p0 = np.random.normal(mean, std, size=(nwalkers, ndim))

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=[luvec, eta, z,C_eta])
%time state = sampler.run_mcmc(p0, 10, progress=True)
sampler.reset()

100%|██████████| 10/10 [01:23<00:00,  8.32s/it]

CPU times: total: 1min 22s
Wall time: 1min 32s


In [44]:
%time sampler.run_mcmc(state.coords, 100, progress=True)
chain = sampler.get_chain()
np.save("100_steps_50_walkers.npy", chain)

100%|██████████| 100/100 [11:05<00:00,  6.65s/it]

CPU times: total: 9min 58s
Wall time: 11min 14s
